In [1]:
import numpy as np
import math
import copy

In [2]:
#Open + read file
a = open('multidim-data.txt', 'r')
sample = a.readlines()
sample = [x.strip('\n') for x in sample] 
sample = [x.split(',') for x in sample] 

#Delete all the attributes you don't care about
for i, asdf in enumerate(sample):
    del(asdf[7])
    del(asdf[0:4])
    
#Turn strings into floats
for i in range(len(sample)):
    for j in range(len(sample[i])):
        sample[i][j] = float(sample[i][j])
        


In [3]:
def hellinger_dist(sampleDist, PDF):
    coefficient = 0
    tempDist = copy.deepcopy(sampleDist)
    totalSum = np.sum(sampleDist)
    for i in indices:
        tempDist[i] = tempDist[i] / totalSum
    for i in indices:
        coefficient += math.sqrt(tempDist[i]*PDF[i])
    return math.sqrt(1-coefficient)

In [4]:
#Recursive helper function to give us all indices so that we can get the values correctly for the multidimensional case.
#You should use this to set a global variable before you need to use it so you only need to call this once. If you call this
#function many times with many dimensions, it will slow down your code a lot.

def nd_range(start, stop, dims):
  if not dims:
    yield ()
    return
  for outer in nd_range(start, stop, dims - 1):
    for inner in range(start, stop):
      yield outer + (inner,)

In [5]:
#This assumes that your sample is currently an array of arrays, each array corresponding to an element and its elements being the
#values of the variables as floats.

#you have to define this global variable before you can run the rest of it.
numBins = 5
    
    
def sample_to_pdf(sample, numObjects): #sample is the sample that we wish to take a subsample from and numobjects is the number
#of groupings we wish to create for the main function.
    m = len(sample[0])
    n = len(sample)
    
    #Compute extrema for binning
    maxValues = np.zeros((m,))
    for i in range(m):
        maxValues[i] = -float('inf')
    for i in sample:
        for j in range(len(i)):
            if i[j] > maxValues[j]:
                maxValues[j] = i[j]
    for i in range(m):
        maxValues[i] += 0.0001

    minValues = np.zeros((m,))
    for i in range(m):
        minValues[i] = float('inf')
    for i in sample:
        for j in range(len(i)):
            if i[j] < minValues[j]:
                minValues[j] = i[j]
    
    

    binSize = (maxValues - minValues) / numBins
    #Create the bins and bin the sample so we get our pdfs as well as the overall pdf.
    distributions = np.zeros((numObjects, numBins, numBins, numBins))
    overallPDF = np.zeros((numBins, numBins, numBins))
    for element in sample:
        i = np.random.randint(0,numObjects)
        
        pos = [0 for i in range(m)]
        for j in range(m):
            pos[j] = math.floor((element[j]-minValues[j]) / binSize[j])
        positions = tuple(pos)
        distributions[i][positions]+= 1
        overallPDF[positions]+=1
    weights = np.zeros((numObjects,))
    for i in range(numObjects):
        weights[i] = np.sum(distributions[i])
    overallPDF = overallPDF / np.sum(overallPDF)
    for i, split in enumerate(distributions):
        tempSum = np.sum(split)
        distributions[i] = split / tempSum
    return weights, distributions, overallPDF

In [6]:
maxWeight = 10000
sampleProportion = 1/5
significanceLevel = 0.9991
w, p, overallPDF =sample_to_pdf(sample, 10000)
m = 3
indices = list(nd_range(0, numBins, m)) #global variable corresponding to all the indices so we only need to generate them once.
#It stores everything as tuples and we just call indices to get indices for multidimensional things since nested for loops don't
#work with an unknown number of dimensions.
def multi_dim_knapsack(w, p, maxWeight):
    n = len(w)
    m = len(p.shape)-1
    totalWeight = 0
    currentPDF = np.zeros([numBins for i in range(m)])
    contents = dict()
    #first, we take a simple random sample that has weight at most maxWeight/2. It should be similar to the population density.
    while True:
        i = np.random.randint(0,n)
        if i not in contents:
            if totalWeight + w[i] > maxWeight*sampleProportion:
                break
            else:
                contents[i] = 1
                totalWeight += w[i]
                totalSum = 0
                for k in indices:
                    totalSum += p[i][k]
                for k in indices:
                    currentPDF[k] += w[i]*p[i][k]/totalSum
    #Now, we add objects to this sample to try to correct it and make it closer to the population density
    
    
    #This is the approach that adds 1 object at a time. This time, we just take anything that's significantly better
    currentDeviation = hellinger_dist(currentPDF, overallPDF)
    pairAdded = True
    numIts=0
    while pairAdded:
        counter = 0
        for i, weight in enumerate(w):
            if i not in contents and totalWeight+w[i] <= maxWeight:
                tempPDF = copy.deepcopy(currentPDF)
                totalSum = 0
                for k in indices:
                    totalSum += p[i][k]
                for k in indices:
                    tempPDF[k] += w[i]*p[i][k]/totalSum
                tempDeviation = hellinger_dist(tempPDF, overallPDF)
                if tempDeviation <= (significanceLevel)*currentDeviation:
                    currentDeviation = tempDeviation
                    totalWeight += w[i]
                    contents[i] = 1
                    currentPDF = tempPDF
                    counter += 1
                    if totalWeight == maxWeight:
                        return contents, currentDeviation, currentPDF, totalWeight, numIts
        if counter == 0:
            pairAdded = False
        numIts += 1
    
    
    return contents, currentDeviation, currentPDF, totalWeight, numIts

In [10]:
a, b, c, d, e = multi_dim_knapsack(w, p, maxWeight)

When running your code on whatever data you are attempting to run this on, the body of the code should look something like this:

(define helper functions above)

(Read in the data as an array of floats or ints)

numBins = k (for a more precise sample, you want to have k as large as possible without making the average number of elements per bin drop too far (ie below 10))

maxWeight = 10000

sampleProportion = 1/5

significanceLevel = 0.9991 (Adjust these 2 parameters manually for best results)

w, p, overallPDF =sample_to_pdf(sample, n).     n is just however many groups of elements you wish to create. Fewer -> faster runtime but too few -> we can't fit many objects into the subsample -> we get a bad subsample

m = 3

indices = list(nd_range(0, numBins, m))

a, b, c, d, e = multi_dim_knapsack(w, p, maxWeight)

a tells you what groups were used in the subsample, which is what you use for whatever you wanted the subsample for.

Finally, a tip about choosing the parameters:

You want to leave sampleProportion somewhere about 0.2 and only increase it if you're struggling to fill your subsample.

To choose significanceLevel, start off with a pretty high value: generally 0.9 for 100 groups of elements and then decrease the distance to 1 by half every time you multiply the number of groups of elements by 10.

Then, you run the algorithm. If the weight of the end result is close to sampleProportion*maxWeight, increase the parameter by a lot. If it's not close to either endpoint, increase it by a little. If the number of iterations (5th variable returned by the function) is 0 or 1, then decrease the parameter by a bit. Repeat this until you get satisfactory results.
